In [1]:
import requests

cards = requests.get("https://c2.scryfall.com/file/scryfall-bulk/oracle-cards/oracle-cards-20201024210538.json").json()

print(cards[1])


{'object': 'card', 'id': '7050735c-b232-47a6-a342-01795bfd0d46', 'oracle_id': '0006faf6-7a61-426c-9034-579f2cfcfa83', 'multiverse_ids': [370780], 'mtgo_id': 49283, 'mtgo_foil_id': 49284, 'tcgplayer_id': 69965, 'cardmarket_id': 262945, 'name': 'Sensory Deprivation', 'lang': 'en', 'released_at': '2013-07-19', 'uri': 'https://api.scryfall.com/cards/7050735c-b232-47a6-a342-01795bfd0d46', 'scryfall_uri': 'https://scryfall.com/card/m14/71/sensory-deprivation?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/7/0/7050735c-b232-47a6-a342-01795bfd0d46.jpg?1562830795', 'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/7/0/7050735c-b232-47a6-a342-01795bfd0d46.jpg?1562830795', 'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/7/0/7050735c-b232-47a6-a342-01795bfd0d46.jpg?1562830795', 'png': 'https://c1.scryfall.com/file/scryfall-cards/png/front/7/0/7050735c-b232-47a6-a342-0179

In [2]:
print(cards[3])

{'object': 'card', 'id': '036ef8c9-72ac-46ce-af07-83b79d736538', 'oracle_id': '000d5588-5a4c-434e-988d-396632ade42c', 'multiverse_ids': [83282], 'mtgo_id': 22609, 'mtgo_foil_id': 22610, 'tcgplayer_id': 12835, 'cardmarket_id': 12551, 'name': 'Storm Crow', 'lang': 'en', 'released_at': '2005-07-29', 'uri': 'https://api.scryfall.com/cards/036ef8c9-72ac-46ce-af07-83b79d736538', 'scryfall_uri': 'https://scryfall.com/card/9ed/100/storm-crow?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/0/3/036ef8c9-72ac-46ce-af07-83b79d736538.jpg?1562730661', 'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/0/3/036ef8c9-72ac-46ce-af07-83b79d736538.jpg?1562730661', 'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/0/3/036ef8c9-72ac-46ce-af07-83b79d736538.jpg?1562730661', 'png': 'https://c1.scryfall.com/file/scryfall-cards/png/front/0/3/036ef8c9-72ac-46ce-af07-83b79d736538.png?156273

In [38]:
def splice_reminders(otext):
    while True:
        start_index = otext.find('(')
        if start_index == -1:
            break
        end_index = otext.find(')')
        otext = otext[0:start_index] + otext[end_index+1:]
    return otext

def extract_data(card):
    if card['layout'] in ["art_series", "double_faced_token", "transform", "split", "adventure", "modal_dfc", "flip"]:
        return None
    if card['set'] in ["unh", "ugl", "ust", "und", 'wc00', 'wc01', 'wc02', 'wc03', 'wc04', 'wc97', 'wc98', 'wc99', 'h17', 'ptg', 'cmb1', 'mznr']:
        #print(card["name"])
        return None
    if "oracle_text" not in card:
        print(card)
    if card["type_line"] == "Card":
        return None
    stats = [
        card["type_line"],
        splice_reminders(card["oracle_text"])
    ]
    if "mana_cost" in card:
        stats.append(card["mana_cost"])
    if "power" in card:
        stats.append(f"power: {card['power']}")
        stats.append(f"toughness: {card['toughness']}")
    if "loyalty" in card:
        stats.append(f"loyalty: {card['loyalty']}")
    
    text = "|".join(stats)
    return text.replace(card['name'], "$").lower()

In [39]:
card_strings = [extract_data(c) for c in cards if extract_data(c) is not None]


In [40]:
longest = max([(len(s), s) for s in card_strings], key=lambda x: x[0])
print(longest)

(619, 'enchantment — aura|enchant creature card in a graveyard\nwhen $ enters the battlefield, if it\'s on the battlefield, it loses "enchant creature card in a graveyard" and gains "enchant creature put onto the battlefield with $." put enchanted creature card onto the battlefield tapped under your control and attach $ to it. when $ leaves the battlefield, that creature\'s controller sacrifices it.\nenchanted creature gets +1/+1 and doesn\'t untap during its controller\'s untap step.\nat the beginning of the upkeep of enchanted creature\'s controller, that player may pay {1}{b}. if the player does, untap that creature.|{1}{b}')


In [15]:
tokens = sorted(set([c for s in card_strings for c in s]))
tokens.insert(0, '<PAD>')

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import numpy as np
from keras_transformer import get_model

In [17]:
model = get_model(
    token_num=len(tokens),
    embed_dim=30,
    encoder_num=3,
    decoder_num=2,
    head_num=3,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((60, 30)),
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Token-Embedding (EmbeddingRet)  [(None, None, 30), ( 1800        Encoder-Input[0][0]              
                                                                 Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 30)     0           Token-Embedding[0][0]            
__________

In [23]:
def encode_card_string(tokens, card_string, pad_length=712):
    return [tokens.index(c) for c in card_string] + [0] * (pad_length - len(card_string))

def encode_card_string_nested(tokens, card_string, pad_length=712):
    return [[tokens.index(c)] for c in card_string] + [[0]] * (pad_length - len(card_string))

def decode_card_string(tokens, enc_card_string):
    return "".join([tokens[i] for i in enc_card_string])

In [24]:
print(max([len(s) for s in card_strings]))

inp = [encode_card_string(tokens, s) for s in card_strings]
outp = [encode_card_string_nested(tokens, s) for s in card_strings]
print(np.asarray(outp).shape)

712
(20922, 712, 1)


In [25]:
model.fit(
    x=[np.asarray(inp), np.asarray(inp)],
    y=np.asarray(outp),
    epochs=5,
)

Epoch 1/5
 5696/20922 [=======>......................] - ETA: 1:17:36 - loss: 0.4415

KeyboardInterrupt: 